# LAB 2

Networks and Systems Security
Week 02
Foundations of Network Security



## Aims of the Seminar

Welcome to the Foundations of Network Security. The aim is to build on the lecture
material of giving you a practical understanding of security needs and technologies.
By the end of this workshop, you will be able to:
• Discuss computer security concepts in practical contexts
• Apply the RASA algorithm to analyse security scenarios
Workshop Outline:
1. RSA Algorithm
2. Socket library
3. Assesment work

In [1]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [2]:
# generate_keys.py
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

# Generate private key
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)

# Save private key
with open("private_key.pem", "wb") as f:
    f.write(
        private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        )
    )
    
# Save public key
public_key = private_key.public_key()
with open("public_key.pem", "wb") as f:
    f.write(
        public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )   
    )
print("✅ Keys saved: private_key.pem, public_key.pem")

✅ Keys saved: private_key.pem, public_key.pem


In [ ]:
# receiver.py
import socket
import pickle
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# Load private key
with open("private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(f.read(), password=None)

# Start server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(("localhost", 65432))
    s.listen()
    print(" Waiting for connection...")
    conn, addr = s.accept()
    with conn:
        print(f" Connected by {addr}")
        data = b""
        while True:
            chunk = conn.recv(4096)
            if not chunk:
                break
            data += chunk

# Unpack payload
encrypted_key, iv, encrypted_message = pickle.loads(data)

# 1. Decrypt AES key with RSA private key
aes_key = private_key.decrypt(
    encrypted_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# 2. Decrypt message with AES
cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
decryptor = cipher.decryptor()
message = decryptor.update(encrypted_message) + decryptor.finalize()

print(" Decrypted message:", message.decode())

 Waiting for connection...
